# Vanilla Steel Case Assessment

### Introduction

This notebook belongs to the solution for case assessment for Vanilla Steel company. In this notebook first we install python libraries, then implement solution for each task. `pandas` library has been used for data manipulation techniques. In each part, I mixed the explanation, code and result to make this notebook easy to follow. Task 1 divided into three steps, first belongs to the supplier_data1 dataframe, then belongs to supplier_data2 dataframe and finally I merge those cleaned dataframes into inventory_dataset based on their shared columns.

### Install & Import Libraries

In [1]:
!pip install -q pandas openpyxl

In [2]:
import os
import pandas as pd

### Initialize Variables

In [3]:
SUPPLIER_1_PATH = './resources/task_1/supplier_data1.xlsx'
SUPPLIER_2_PATH = './resources/task_1/supplier_data2.xlsx'
REFERENCE_PROPERTIES_PATH = './resources/task_2/reference_properties.tsv'
RFQ_PATH = './resources/task_2/rfq.csv'
OUTPUT_FOLDER_PATH = './output/'

In [4]:
sup_1_df = pd.read_excel(SUPPLIER_1_PATH)
sup_2_df = pd.read_excel(SUPPLIER_2_PATH)
ref_df = pd.read_csv(REFERENCE_PROPERTIES_PATH, sep='\t')
rfq_df = pd.read_csv(RFQ_PATH)

## Task 1

In this task we are going to first clean and normalize each dataframe, then merge two dataframes together.

### Task A.1

First we check the head of dataframes, information about each column and check the missing values.

#### Supplier 1

In [5]:
sup_1_df.head()

,Quality/Choice,Grade,Finish,Thickness (mm),Width (mm),Description,Gross weight (kg),RP02,RM,Quantity,AG,AI
0,3rd,C200S,gebeizt und geglüht,2.77,1100,Längs- oder Querisse,13983,333.6,606.2,0.00,16.11,0.0054
1,3rd,C300S,ungebeizt,2.65,1075,Längs- oder Querisse,13047,717.7,0.0,0.00,16.11,0.0046
2,3rd,C100S,gebeizt und geglüht,2.20,1100,Kantenfehler - FS-Kantenrisse,14155,368.9,0.0,10.84,0.00,0.0061
3,2nd,C100S,gebeizt,2.86,1100,Längs- oder Querisse,11381,368.9,601.7,22.87,0.00,0.0062
4,1st,C300S,ungebeizt,2.88,1050,Sollmasse (Gewicht) unterschritten,10072,0.0,1213.0,22.87,0.00,0.0041


In [6]:
sup_1_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Quality/Choice     50 non-null     object 
 1   Grade              50 non-null     object 
 2   Finish             50 non-null     object 
 3   Thickness (mm)     50 non-null     float64
 4   Width (mm)         50 non-null     int64  
 5   Description        50 non-null     object 
 6   Gross weight (kg)  50 non-null     int64  
 7   RP02               50 non-null     float64
 8   RM                 50 non-null     float64
 9   Quantity           50 non-null     float64
 10  AG                 50 non-null     float64
 11  AI                 50 non-null     float64
dtypes: float64(6), int64(2), object(4)
memory usage: 4.8+ KB


After checking the above information, I noticed that there is no null values into dataframe, and these actions should be done :

<ul>
    <li>Apply strip, lowe or upper functions on string columns based on column entity</li>
    <li>Apply type of <b>Thickness (mm)</b>, <b>Width (mm)</b> and <b>Gross weight (kg)</b> to be float64</li>
    <li>Rename column <b>Gross weight (kg)</b> to <b>Weight (kg)</b></li>
    <li>Assign supplier name and unique id for each row for further checking, the unique id helps us later to get detailed information later.</li>
</ul>
</br>

In [7]:
sup_1_df['Quality/Choice'] = sup_1_df['Quality/Choice'].str.lower().str.strip()
sup_1_df['Finish'] = sup_1_df['Finish'].str.lower().str.strip()
sup_1_df['Grade'] = sup_1_df['Grade'].str.upper().str.strip()
sup_1_df['Description'] = sup_1_df['Description'].str.strip()

In [8]:
for column in ['Thickness (mm)', 'Width (mm)', 'Gross weight (kg)', 'Quantity']:
    sup_1_df[column] = sup_1_df[column].astype(float)

In [9]:
sup_1_df = sup_1_df.rename(columns={
    'Gross weight (kg)': 'Weight (kg)'
})

In [10]:
sup_1_df['Supplier'] = 'Supplier 1'
sup_1_df['Article ID'] = range(1, len(sup_1_df) + 1)

In [11]:
sup_1_df.head()

,Quality/Choice,Grade,Finish,Thickness (mm),Width (mm),Description,Weight (kg),RP02,RM,Quantity,AG,AI,Supplier,Article ID
0,3rd,C200S,gebeizt und geglüht,2.77,1100.0,Längs- oder Querisse,13983.0,333.6,606.2,0.00,16.11,0.0054,Supplier 1,1
1,3rd,C300S,ungebeizt,2.65,1075.0,Längs- oder Querisse,13047.0,717.7,0.0,0.00,16.11,0.0046,Supplier 1,2
2,3rd,C100S,gebeizt und geglüht,2.20,1100.0,Kantenfehler - FS-Kantenrisse,14155.0,368.9,0.0,10.84,0.00,0.0061,Supplier 1,3
3,2nd,C100S,gebeizt,2.86,1100.0,Längs- oder Querisse,11381.0,368.9,601.7,22.87,0.00,0.0062,Supplier 1,4
4,1st,C300S,ungebeizt,2.88,1050.0,Sollmasse (Gewicht) unterschritten,10072.0,0.0,1213.0,22.87,0.00,0.0041,Supplier 1,5


#### Supplier 2

In [12]:
sup_2_df.head()

,Material,Description,Article ID,Weight (kg),Quantity,Reserved
0,HDC,Material is Oiled,23048203,24469,52,NOT RESERVED
1,S235JR,Material is Oiled,23040547,16984,41,NOT RESERVED
2,S235JR,Material is Painted,23046057,9162,28,NOT RESERVED
3,DX51D +AZ150,Material is Oiled,23041966,12119,66,VANILLA
4,HDC,Material is Painted,23043884,17260,26,NOT RESERVED


In [13]:
sup_2_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Material     50 non-null     object
 1   Description  50 non-null     object
 2   Article ID   50 non-null     int64 
 3   Weight (kg)  50 non-null     int64 
 4   Quantity     50 non-null     int64 
 5   Reserved     50 non-null     object
dtypes: int64(3), object(3)
memory usage: 2.5+ KB


Again for the second dataframe after checking the above information, I noticed that there is no null values into dataframe, and these actions should be done :

<ul>
    <li>Apply strip, lower or upper functions on string columns based on column entity</li>
    <li>Apply type of <b>Weight (kg)</b> and <b>Quantity</b> to be float64</li>
    <li>Rename column <b>Material</b> to <b>Grade</b> to keep consistency for mergeing dataframes</li>
    <li>Assign supplier name for each row for further checking</li>
</ul>
</br>

In [14]:
sup_2_df['Material'] = sup_2_df['Material'].str.strip().str.upper()
sup_2_df['Reserved'] = sup_2_df['Reserved'].str.strip().str.upper()

In [15]:
for column in ['Weight (kg)', 'Quantity']:
    sup_2_df[column] = sup_2_df[column].astype(float)

In [16]:
sup_2_df = sup_2_df.rename(columns={
    'Material': 'Grade'
})

In [17]:
sup_2_df['Supplier'] = 'Supplier 2'

In [18]:
sup_2_df.head()

,Grade,Description,Article ID,Weight (kg),Quantity,Reserved,Supplier
0,HDC,Material is Oiled,23048203,24469.0,52.0,NOT RESERVED,Supplier 2
1,S235JR,Material is Oiled,23040547,16984.0,41.0,NOT RESERVED,Supplier 2
2,S235JR,Material is Painted,23046057,9162.0,28.0,NOT RESERVED,Supplier 2
3,DX51D +AZ150,Material is Oiled,23041966,12119.0,66.0,VANILLA,Supplier 2
4,HDC,Material is Painted,23043884,17260.0,26.0,NOT RESERVED,Supplier 2


#### Merge

In this section, we merge the both dataframes based on share columns and save inventory_dataset.csv file.

In [19]:
shared_columns = sup_1_df.columns.intersection(sup_2_df.columns)
print("Shared columns:", shared_columns.tolist())

Shared columns: ['Grade', 'Description', 'Weight (kg)', 'Quantity', 'Supplier', 'Article ID']


In [20]:
sup_1_shared = sup_1_df[shared_columns]
sup_2_shared = sup_2_df[shared_columns]
inventory_dataset = pd.concat([sup_1_shared, sup_2_shared], ignore_index = True)

Becuase the inventory dataset belongs to the available steels, the we filter rows that has a zero quantity.

In [21]:
inventory_dataset = inventory_dataset[inventory_dataset['Quantity'] > 0]

In [22]:
inventory_dataset.sample(frac=1).head()

,Grade,Description,Weight (kg),Quantity,Supplier,Article ID
62,S235JR,Material is not Oiled,15881.0,36.00,Supplier 2,23047348
6,C100S,Sollmasse (Gewicht) unterschritten,8936.0,10.84,Supplier 1,7
42,C100S,Sollmasse (Gewicht) unterschritten,11360.0,22.87,Supplier 1,43
98,DX51D +Z140,Material is Oiled,16927.0,69.00,Supplier 2,23043348
63,HDC,Material is Painted,23915.0,30.00,Supplier 2,23045961


In [23]:
os.makedirs(OUTPUT_FOLDER_PATH, exist_ok=True)
inventory_dataset.to_csv(os.path.join(OUTPUT_FOLDER_PATH, 'inventory_dataset.csv'), index=False)